## Notebook setup

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Dependencies

In [ ]:
!pip install -r "/content/drive/MyDrive/ml_projects/tdt12_nlp_creative/requirements.txt"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 15.0 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 708 kB 60.0 MB/s 
     |████████████████████████████████| 572 kB 64.5 MB/s 
     |████████████████████████████████| 529 kB 57.2 MB/s 
     |████████████████████████████████| 182 kB 70.4 MB/s 
     |████████████████████████████████| 7.6 MB 55.4 MB/s 
     |████████████████████████████████| 87 kB 8.4 MB/s 
  Created wheel for aitextgen: filename=aitextgen-0.6.0-py3-none-any.whl size=576010 sha256=219d222a758d19d2cf9c33e4d380b4e68e3a0177f857f7b388c2f25d5c4a8ec6
  Stored in directory: /root/.cache/pip/wheels/eb/77/2a/993dcd2bf690ff75db487709aa0f770f15ea57a7f8ed088524
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=a35f95e77572c9c1e02723e54aeeed047b36f10dbb3acda71f75184efae7afb4
  Stored in directory: /ro

### Python Imports

In [ ]:
# File/string operations
DATA = "/content/drive/MyDrive/ml_projects/tdt12_nlp_creative"
import shutil
import os
# Logging
import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )
# Model operations
from aitextgen import aitextgen

### Verify active GPU in Colab

In [ ]:
!nvidia-smi

Wed Nov 23 16:31:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load TokenDataset

In [ ]:
file_name = "dataset_cache.tar.gz"
shutil.copy(f"{DATA}/dataset/{file_name}", f"./{file_name}")

'./dataset_cache.tar.gz'

## Model fine-tuning

### Load Model


There are several sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M` (default): the "medium" model, 1.5GB on disk.
* `774M` (default): the "large" model, 3GB on disk.

You can also finetune a GPT Neo model instead, which is more suitable for longer texts and the base model has more recent data:

* `125M`: Analogous to the GPT-2 124M model.
* `350M`: Analogous to the GPT-2 355M model

The next cell downloads the model and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

In [ ]:
ai = aitextgen(tf_gpt2="355M", to_gpu=True)

# Comment out the above line and uncomment the below line to use GPT Neo instead.
# ai = aitextgen(model="EleutherAI/gpt-neo-125M", to_gpu=True)

INFO:aitextgen:Downloading the 355M GPT-2 TensorFlow weights/config from Google's servers


Fetching checkpoint:   0%|          | 0.00/77.0 [00:00<?, ?it/s]

Fetching hparams.json:   0%|          | 0.00/91.0 [00:00<?, ?it/s]

Fetching model.ckpt.data-00000-of-00001:   0%|          | 0.00/1.42G [00:00<?, ?it/s]

Fetching model.ckpt.index:   0%|          | 0.00/10.4k [00:00<?, ?it/s]

...

INFO:aitextgen:Converting the 355M GPT-2 TensorFlow weights to PyTorch.
Converting TensorFlow checkpoint from /content/aitextgen/355M
Loading TF weight model/h0/attn/c_attn/b with shape [3072]
Loading TF weight model/h0/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h0/attn/c_proj/b with shape [1024]
Loading TF weight model/h0/attn/c_proj/w with shape [1, 1024, 1024]
Loading TF weight model/h0/ln_1/b with shape [1024]
Loading TF weight model/h0/ln_1/g with shape [1024]
Loading TF weight model/h0/ln_2/b with shape [1024]
Loading TF weight model/h0/ln_2/g with shape [1024]
Loading TF weight model/h0/mlp/c_fc/b with shape [4096]
Loading TF weight model/h0/mlp/c_fc/w with shape [1, 1024, 4096]
Loading TF weight model/h0/mlp/c_proj/b with shape [1024]
Loading TF weight model/h0/mlp/c_proj/w with shape [1, 4096, 1024]
Loading TF weight model/h1/attn/c_attn/b with shape [3072]
Loading TF weight model/h1/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h1/attn

Save PyTorch model to aitextgen/pytorch_model.bin


INFO:aitextgen:Loading 355M GPT-2 model from /aitextgen.


Save configuration file to aitextgen/config.json


INFO:aitextgen:GPT2 loaded with 354M parameters.
INFO:aitextgen:Gradient checkpointing enabled for model training.
INFO:aitextgen:Using the default GPT-2 Tokenizer.


### Fine-tuning

In [ ]:
ai.train(file_name,
         line_by_line=True,
         from_cache=True,
         num_steps=5000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=True,
         learning_rate=1e-3,
         batch_size=1, 
         )

INFO:aitextgen:Loading text from dataset_cache.tar.gz with generation length of 1024.
INFO:aitextgen.TokenDataset:TokenDataset containing 19,287,928 subsets loaded via cache.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:259: LightningDeprecationWarning: The `Callback.on_batc

  0%|          | 0/5000 [00:00<?, ?it/s]

1,000 steps reached: saving model to /trained_model


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


1,000 steps reached: generating sample texts.
, but it also works, and the film has been a great, great film. The film is great. It has a lot of moments, and it's a great movie. It is a great film, but it does not follow the book. The acting is great, the story starts as a beautiful house with the main character. It is too long, as well as the film itself. It is the film. The plot is a little boring, the film is very predictable and very predictable, the story has some very funny moments, but the acting is very strange. The acting is very good, the writing is over-dramatise, the film has lots of moments where the film is shown in a musical hall and a musical hall. It's a good movie, but the direction is very bad. The film may not be a good movie, but it's still a very cheesy one. And you could't imagine what other is making. It's a good movie, and if you don't go to see it, you're not expecting to be disappointed."

2,000 steps reached: saving model to /trained_model


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


5,000 steps reached: generating sample texts.


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=5000` reached.


 was not really too sure what to do with the titular character. It was so much more like a big disappointment. It was in the sense that it would be the worst movie we've ever seen in a while. How can you get an answer to what you have already just just watched? If you're going to spend time with this movie, you might have a better time with this movie. I'm glad I did. I can't blame the acting. I thought the actors who gave an excellent job did not make a good job. They did not make a perfect job for the movie. I'm sure they could have given the movie a 7.0. I'm sure there were a few people who had no idea what to do with the character. I don't know what to do with their actions. The movie just didn't work. I can relate to them. They seemed to tell the story that they could have done it too well. I don't know what to do with those two. I'm sure if they were just trying to make a movie about what would happen with the other characters. Then they built a lot of suspense. I'll just assume 

INFO:aitextgen:Saving trained model pytorch_model.bin to /trained_model


## Inference

### Re-load model for increased performance while inferencing

In [ ]:
ai = aitextgen(model_folder="trained_model", to_gpu=True)

INFO:aitextgen:Loading model from provided weights and config in /trained_model.
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:370: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
INFO:aitextgen:GPT2 loaded with 354M parameters.
INFO:aitextgen:Using the default GPT-2 Tokenizer.


### Inference for each rating

In [ ]:
reviews = []
for i in range(1,11):
  reviews.append(ai.generate(n=5,
                             prompt=f"{i}<|rating|>",
                             prepend_bos=None,
                             min_length=None,
                             max_length=500,
                             temperature=0.8,
                             top_k=40, # Limits sampled tokens to the top k values
                             #top_p=40, # Limits sampled tokens to the cumulative probability
                             #num_beams=5, # If greater than 1, executes beam search for cleaner text
                             #repetition_penalty=1.0, # If greater than 1.0, penalizes repetition in a text to avoid infinite loops
                             #length_penalty=1.0, # If greater than 1.0, penalizes text proportional to the length
                             do_sample=True,
                             return_as_list=True,
                             ))

for i, rating in enumerate(reviews):
  rating_text = "-"*15 + f"\n  Rating: {i+1}\n" + "-"*15
  print(rating_text)
  for j, review in enumerate(rating):
    print("'" + review.split("<|rating|>")[-1] + "'")
  print()

---------------
  Rating: 1
---------------
'I do not really believe that this film was ever seen on video. It was simply an insult to the intelligence of the main characters.If I had a problem with this movie I would have no problem with the film. The story and atmosphere of the film had absolutely nothing to do with the movie. I was glad to see what I could do with the film. I was laughing at the movie, and I did not have the same feeling as the film. And it's pretty clear that it was a very poorly executed setup. Don't blame the actors for this movie. They should be ashamed to be ashamed for the poor acting. The music was very badly written and the sound was horrible. I can only think about how much better it was. The film is a lot of the film. I didn't know what to do with the plot. It seemed to be a nod to films like the 'Killer and the Bird' and the 'Killer. I cannot deny this. I have no idea what director/actor/actor/FX was thinking about. I can't believe what people were thinki